[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap04/cap04-main.ipynb)

# Capítulo 4 - Do Monte Carlo ao Q-Learning


Aqui, veremos dois algoritmos de tipos distintos para o problema de *controle* da aprendizagem por reforço -- ou seja, o problema de aprender a política ótima.

O principal que eles têm em comum, é o fato de que **eles se baseiam na função de valor da ação `Q(s,a)`**. Esta função é calculada e usada durante a execução de ambos os algoritmos.

Relembrando, o $Q(s,a)$ ele é o valor médio das recompensas parciais a partir do par $(s,a)$:

$$Q(s,a) = E[G_t | S_t=s, A_t=a]$$

De maneira informa, ele responde a esta pergunta:

*Quando estava no estado* **s** *e fez a ação* **a** *, qual o retorno esperado (se continuar seguindo a política no restante do episódio)?*

Vamos ver como usar esta informação para fazer a escolha de boas ações!

In [ ]:
from IPython.display import clear_output
import sys

if 'google.colab' in sys.modules:
    !pip install gym==0.23.1
    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")
    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [ ]:
import gym
import numpy as np

from util.plot import plot_result
from util.experiments import test_greedy_Q_policy

In [ ]:
# vamos focar nesses três ambientes por serem mais simples
# ver mais em: https://www.gymlibrary.dev/
#env = gym.make("Taxi-v3")
env = gym.make("FrozenLake-v1")
#env = gym.make("Blackjack-v1")

## 1 - Algoritmo (de Controle) de Monte Carlo - Versão 1

Este algoritmo roda vários episódios, fazendo estes passos a cada episódio:

1. Gera a **trajetória** completa (sequência de estados/observações, ações e recompensas) do episódio:

    $S_0 \rightarrow A_0 \rightarrow R_1 \rightarrow S_1 \rightarrow A_1 \rightarrow R_2 \rightarrow S_2 \rightarrow \cdots S_{n-1} \rightarrow A_{n-1} \rightarrow R_T \rightarrow S_T$

1. Para escolher a ação $a$, a ser realizada em um estado $s$, ele usa a tabela $Q(s,a)$ com alguma estratégia de exploração -- vamos usar $\epsilon$-greedy. Assim, ele escolhe dessa forma:
   - com probabilidade $\epsilon$, ele escolhe uma ação $a$ qualquer
   - com probabilidade $1-\epsilon$, ele escolhe a melhor ação, ou seja, $\max_a{Q(s,a)}$

1. Ao fim do episódio, para cada para intermediário ($S_t$, $A_t$) da trajetória, ele:
   - calcula o retorno parcial $G_t$.
   - usa $G_t$ para atualizar $Q(S_t, A_t)$


In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def choose_action_1(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])   # alt. para aleatorizar empates: np.random.choice(np.where(b == bmax)[0])


# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarlo1(env, episodes, gamma=0.95, epsilon=0.1, render=False):
    num_actions = env.action_space.n

    # dicionário com todos os retornos descontados, para cada par (estado,ação)
    returns_history = dict()

    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []

        state = env.reset()

        # [1] Executa um episódio completo, salvando a trajetória
        while not done:
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()

            # [2] Escolhe a próxima ação -- usa epsilon-greedy
            action = choose_action_1(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # [3] Calcula os retornos parciais e atualiza Q
        Gt = 0
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt

            if returns_history.get((s,a)) is None:
                returns_history[s,a] = [ Gt ]
            else:
                returns_history[s,a].append(Gt)

            # média entre todas as ocorrências de (s,a) encontradas nos episódios
            Q[s,a] = np.mean(returns_history[s,a]) # LENTO! -> vamos melhorar
            # cálculo alternativo da média: M = M + (1/n)*(x - M)

    return sum_rewards_per_ep, Q


In [ ]:
%timeit
ENV_NAME = "Taxi-v3"
#ENV_NAME = "Blackjack-v1"
r_max_plot = 10

EPISODES = 3000
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME)

# Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
rewards, Qtable = run_montecarlo1(env, EPISODES, GAMMA, EPSILON, render=False)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

# Mostra um gráfico de episódios x retornos (não descontados)
# Se quiser salvar, passe o nome do arquivo no 3o parâmetro
filename = f"results/montecarlo1-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
plot_result(rewards, r_max_plot, None)

## 2 - Algoritmo (de Controle) de Monte Carlo - Versão 2

Modifique o código acima com essas características:
1. Mantenha o uso de `choose_action_1()` para escolher a ação.
1. Remova o **histórico** de retornos parciais.
1. Use uma **taxa de aprendizagem**, representada pelo parâmetro `lr`.

Faça as melhorias abaixo:

In [ ]:
# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarlo2(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []

        state = env.reset()

        # [1] Executa um episódio completo, salvando a trajetória
        while not done:
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()

            # [2] Escolhe a próxima ação -- usa epsilon-greedy
            action = choose_action_1(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # [3] Calcula os retornos parciais e atualiza Q
        Gt = 0
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt
            # FAÇA AQUI A MODIFICAÇÂO PEDIDA !!!

    return sum_rewards_per_ep, Q

Quando o código estiver pronto, você poderá testar rodando o código abaixo:

In [ ]:
ENV_NAME = "FrozenLake-v1" #"Taxi-v3"
r_max_plot = 1.0

EPISODES = 20000
LR = 0.01
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME)

# Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
rewards, Qtable = run_montecarlo2(env, EPISODES, LR, GAMMA, EPSILON, render=False)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

# Mostra um gráfico de episódios x retornos não descontados
# Se quiser salvar, passe o nome do arquivo no 3o parâmetro
#filename = f"results/montecarlo2-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
plot_result(rewards, r_max_plot, None)

## 3 - Algoritmo Q-Learning

Explicar um pouco aqui...

In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def choose_action_2(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        # melhoria: se houver empate, desempata aleatoriamente
        max_q = np.max(Q[state])
        return np.random.choice(np.where(Q[state] == max_q)[0])


# Algoritmo Q-learning, online learning (TD-learning)
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_qlearning(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.random.uniform(low = -1.0, high = 0.0,
                          size = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state = env.reset()

        # executa um episódio completo, fazendo atualizações na Q-table
        while not done:
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()

            # escolhe a próxima ação -- usa epsilon-greedy
            action = choose_action_2(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            if done:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table
            # delta = (estimativa usando a nova recompensa) - estimativa antiga
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            sum_rewards += reward
            state = next_state

        #epsilon = np.exp(-0.005*i)

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_ep, Q

Agora, vamos testar o algoritmo:

In [ ]:
ENV_NAME = "Taxi-v3"
r_max_plot = 10

EPISODES = 20000
LR = 0.01
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME)

# Roda o algoritmo Q-Learning
rewards, Qtable = run_qlearning(env, EPISODES, LR, GAMMA, EPSILON, render=False)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

# Mostra um gráfico de episódios x retornos não descontados
# Se quiser salvar, passe o nome do arquivo no 3o parâmetro
#filename = f"results/qlearning-{ENV_NAME.lower()[0:8]}-ep{EPISODES}-lr{LR}.png"
plot_result(rewards, r_max_plot, None)

In [ ]:
test_greedy_Q_policy(env, Qtable, 10, False)

## 4 - Experimentos Finais


In [ ]:
from util.experiments import repeated_exec
from util.plot import plot_multiple_results

Vamos comparar os algoritmos aqui propostos.

In [ ]:
NUM_EPISODES = 12000

enviroment = gym.make("Taxi-v3")
#enviroment = gym.make("FrozenLake-v1")

results = []
# muito lento!
results.append( repeated_exec(1, "Monte-Carlo1", run_montecarlo1, enviroment, NUM_EPISODES) )

#for learning_rate in [0.01, 0.1, 0.5]:
#    results.append( repeated_exec(1, f"Monte-Carlo2 (LR={learning_rate})", run_montecarlo2, enviroment, NUM_EPISODES, learning_rate) )

for learning_rate in [0.05, 0.1, 0.5, 1.0]:
    results.append( repeated_exec(1, f"Q-Learning (LR={learning_rate})", run_qlearning, enviroment, NUM_EPISODES, learning_rate) )

clear_output()

plot_multiple_results(results, cumulative=False, x_log_scale=False)